# 1. Labraries Loading

In [67]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

# 2. Dataset opening

In [68]:
measurementsDf = pd.read_json('../../../../Data/RawData/measurements.json')

In [69]:
measurementsDf.head(10)

,time,T,Rn,H,P,CO2,sensor_id
0,2020-06-27T10:00:00.000Z,3.000000,NaN,14.000000,NaN,NaN,@0123
1,2020-07-14T14:00:00.000Z,31.300000,NaN,44.475000,NaN,567.750000,D0001
2,2020-07-14T15:00:00.000Z,31.333333,NaN,45.505556,NaN,636.361111,D0001
3,2020-07-14T16:00:00.000Z,31.451429,NaN,47.145714,NaN,734.000000,D0001
4,2020-07-14T17:00:00.000Z,31.161765,NaN,36.000000,NaN,461.529412,D0001
5,2020-07-14T18:00:00.000Z,31.008333,NaN,28.436111,NaN,400.000000,D0001
6,2020-07-14T19:00:00.000Z,30.647222,NaN,29.133333,NaN,400.000000,D0001
7,2020-07-14T20:00:00.000Z,29.891429,NaN,28.597143,NaN,400.000000,D0001
8,2020-07-14T21:00:00.000Z,29.118919,NaN,29.162162,NaN,400.000000,D0001
9,2020-07-14T22:00:00.000Z,28.200000,NaN,31.611111,NaN,400.000000,D0001


In [62]:
discrete_features = [col for col in measurementsDf.columns if measurementsDf[col].dtype != object]

In [63]:
discrete_features

['T', 'Rn', 'H', 'P', 'CO2']

In [4]:
print(measurementsDf.shape)

(117274, 7)


# 3. Data Preprocessing

In [5]:
# Its important to check for the null values

measurementsDf.isnull().sum()

time             0
T            38122
Rn            3160
H            38106
P            30649
CO2           3851
sensor_id        0
dtype: int64

In [122]:
# Null values visualization
#measurementsDf = measurementsDf.drop(['time'],axis = 1)
measurementsDf[measurementsDf['T'].isnull()]

,time,T,Rn,H,P,CO2,sensor_id


## 3.1 Data inputation using mean, median and most frequent from sklearn input librabry

### Mean inputation

In [7]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')

measurementsDf['T'] =  imputer.fit_transform(measurementsDf[["T"]]).ravel()
measurementsDf['Rn'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['H'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['P'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['CO2'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()

measurementsDf.head(15)

,time,T,Rn,H,P,CO2,sensor_id
0,2020-06-27T10:00:00.000Z,3.000000,648.532105,648.532105,648.532105,648.532105,@0123
1,2020-07-14T14:00:00.000Z,31.300000,648.532105,648.532105,648.532105,648.532105,D0001
2,2020-07-14T15:00:00.000Z,31.333333,648.532105,648.532105,648.532105,648.532105,D0001
3,2020-07-14T16:00:00.000Z,31.451429,648.532105,648.532105,648.532105,648.532105,D0001
4,2020-07-14T17:00:00.000Z,31.161765,648.532105,648.532105,648.532105,648.532105,D0001
5,2020-07-14T18:00:00.000Z,31.008333,648.532105,648.532105,648.532105,648.532105,D0001
6,2020-07-14T19:00:00.000Z,30.647222,648.532105,648.532105,648.532105,648.532105,D0001
7,2020-07-14T20:00:00.000Z,29.891429,648.532105,648.532105,648.532105,648.532105,D0001
8,2020-07-14T21:00:00.000Z,29.118919,648.532105,648.532105,648.532105,648.532105,D0001
9,2020-07-14T22:00:00.000Z,28.200000,648.532105,648.532105,648.532105,648.532105,D0001


### Median inputation

In [85]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')

measurementsDf['T'] =  imputer.fit_transform(measurementsDf[["T"]]).ravel()
measurementsDf['Rn'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['H'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['P'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['CO2'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()

measurementsDf.head(15)

,time,T,Rn,H,P,CO2,sensor_id
0,2020-06-27T10:00:00.000Z,3.000000,648.532105,648.532105,648.532105,648.532105,@0123
1,2020-07-14T14:00:00.000Z,31.300000,648.532105,648.532105,648.532105,648.532105,D0001
2,2020-07-14T15:00:00.000Z,31.333333,648.532105,648.532105,648.532105,648.532105,D0001
3,2020-07-14T16:00:00.000Z,31.451429,648.532105,648.532105,648.532105,648.532105,D0001
4,2020-07-14T17:00:00.000Z,31.161765,648.532105,648.532105,648.532105,648.532105,D0001
5,2020-07-14T18:00:00.000Z,31.008333,648.532105,648.532105,648.532105,648.532105,D0001
6,2020-07-14T19:00:00.000Z,30.647222,648.532105,648.532105,648.532105,648.532105,D0001
7,2020-07-14T20:00:00.000Z,29.891429,648.532105,648.532105,648.532105,648.532105,D0001
8,2020-07-14T21:00:00.000Z,29.118919,648.532105,648.532105,648.532105,648.532105,D0001
9,2020-07-14T22:00:00.000Z,28.200000,648.532105,648.532105,648.532105,648.532105,D0001


### Iteratve imputation

In [64]:
iterative_imputater = IterativeImputer()

In [65]:
iter_imputer = IterativeImputer(random_state=42)
iter_imputed = iter_imputer.fit_transform(measurementsDf[discrete_features])
df_iter_imputed = pd.DataFrame(iter_imputed, columns=discrete_features)
#iterative_imputater.fit_transform(measurementsDf)

In [66]:
df_iter_imputed

,T,Rn,H,P,CO2
0,3.000000,2404.663876,14.000000,102.643690,150.232208
1,31.300000,448.456100,44.475000,101.775235,567.750000
2,31.333333,417.807923,45.505556,101.786788,636.361111
3,31.451429,365.207804,47.145714,101.801547,734.000000
4,31.161765,775.108082,36.000000,101.719291,461.529412
...,...,...,...,...,...
117269,30.545190,1725.951292,11.210526,101.557905,88.493014
117270,30.816691,1827.157380,8.500000,101.531039,87.653993
117271,30.766608,1808.488296,9.000000,101.535995,87.808764
117272,30.000000,262.134392,48.000000,101.800679,92.869113


### KNN imputation

In [70]:
knn_imputer = KNNImputer(n_neighbors=5, add_indicator=True)

In [75]:
#measurementsDf = measurementsDf.drop(['time','sensor_id'],axis = 1)
#knn_imputer.fit_transform(measurementsDf)
measurementsDf[discrete_features].head(10)

,T,Rn,H,P,CO2
0,3.000000,NaN,14.000000,NaN,NaN
1,31.300000,NaN,44.475000,NaN,567.750000
2,31.333333,NaN,45.505556,NaN,636.361111
3,31.451429,NaN,47.145714,NaN,734.000000
4,31.161765,NaN,36.000000,NaN,461.529412
5,31.008333,NaN,28.436111,NaN,400.000000
6,30.647222,NaN,29.133333,NaN,400.000000
7,29.891429,NaN,28.597143,NaN,400.000000
8,29.118919,NaN,29.162162,NaN,400.000000
9,28.200000,NaN,31.611111,NaN,400.000000


In [78]:
knn_imputer.fit(measurementsDf[discrete_features])

KNNImputer(add_indicator=True)

In [79]:
knn_imputer.transform(measurementsDf[discrete_features])

array([[  3.        ,   6.84527485,  14.        , ...,   0.        ,
          1.        ,   1.        ],
       [ 31.3       ,  18.752     ,  44.475     , ...,   0.        ,
          1.        ,   0.        ],
       [ 31.33333333,  26.80018022,  45.50555556, ...,   0.        ,
          1.        ,   0.        ],
       ...,
       [  4.7395619 ,  11.30753158,   9.        , ...,   0.        ,
          1.        ,   1.        ],
       [ 30.        ,  77.00983333,  48.        , ...,   0.        ,
          1.        ,   1.        ],
       [ 30.        , 148.589     ,  53.        , ...,   0.        ,
          1.        ,   1.        ]])

In [82]:
pd.DataFrame(knn_imputer.transform(measurementsDf[discrete_features])).head(20)

,0,1,2,3,4,5,6,7,8,9
0,3.000000,6.845275,14.000000,97.732064,49.966374,0.0,1.0,0.0,1.0,1.0
1,31.300000,18.752000,44.475000,102.106461,567.750000,0.0,1.0,0.0,1.0,0.0
2,31.333333,26.800180,45.505556,101.620167,636.361111,0.0,1.0,0.0,1.0,0.0
3,31.451429,15.380598,47.145714,101.684748,734.000000,0.0,1.0,0.0,1.0,0.0
4,31.161765,15.359386,36.000000,101.611684,461.529412,0.0,1.0,0.0,1.0,0.0
5,31.008333,10.021789,28.436111,101.477564,400.000000,0.0,1.0,0.0,1.0,0.0
6,30.647222,13.527832,29.133333,101.794481,400.000000,0.0,1.0,0.0,1.0,0.0
7,29.891429,9.044926,28.597143,101.381345,400.000000,0.0,1.0,0.0,1.0,0.0
8,29.118919,7.726126,29.162162,101.777480,400.000000,0.0,1.0,0.0,1.0,0.0
9,28.200000,16.050054,31.611111,101.503572,400.000000,0.0,1.0,0.0,1.0,0.0


In [81]:
measurementsDf.isnull().sum()

time             0
T            38122
Rn            3160
H            38106
P            30649
CO2           3851
sensor_id        0
dtype: int64

## 3.2 Data inputation using datawig library

In [50]:
knn_imputer = KNNImputer()

In [ ]:
measurementsDf.head(20)